In [1]:
import quandl
import pickle
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
def get_quandl_data(quandl_id):
    '''Download and cache Quandl dataseries'''
    cache_path = '{}.pkl'.format(quandl_id).replace('/','-')
    try:
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(quandl_id))
    except (OSError, IOError) as e:
        print('Downloading {} from Quandl'.format(quandl_id))
        df = quandl.get(quandl_id, returns="pandas")
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(quandl_id, cache_path))
    return df

In [3]:
exchanges = ['KRAKEN','COINBASE','BITSTAMP','ITBIT']

exchanges_list = []
for exchange in exchanges:
    exchange_code = 'BCHARTS/{}USD'.format(exchange)
    df_exchange_temp = get_quandl_data(exchange_code)
    df_exchange_temp.reset_index(inplace=True)
    df_exchange_temp = df_exchange_temp.loc[(df_exchange_temp['Date'] >= datetime(2015, 12, 1)) & 
                                            (df_exchange_temp['Date'] < datetime(2018,2,1))]
    
    exchanges_list.append(df_exchange_temp)

Loaded BCHARTS/KRAKENUSD from cache
Loaded BCHARTS/COINBASEUSD from cache
Loaded BCHARTS/BITSTAMPUSD from cache
Loaded BCHARTS/ITBITUSD from cache


In [4]:
df_total = pd.concat(exchanges_list)
df_total.sort_values('Date', inplace=True)
df_total.reset_index(drop=True, inplace=True)
df_total.head(16)

,Date,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
0,2015-12-01,377.68718,378.4700,355.00000,361.58200,442.081445,1.611765e+05,364.585453
1,2015-12-01,376.70000,379.0000,354.56000,363.06000,15410.282352,5.617537e+06,364.531747
2,2015-12-01,377.03000,378.9600,355.09000,363.31000,6814.018100,2.482811e+06,364.368183
3,2015-12-01,376.88000,378.9400,354.60000,362.68000,9267.092086,3.387642e+06,365.556118
4,2015-12-02,362.40660,362.4066,349.20001,358.69000,228.556211,8.130658e+04,355.739952
5,2015-12-02,362.70000,362.8400,348.64000,358.89000,12343.407768,4.390063e+06,355.660512
6,2015-12-02,362.68000,363.0600,347.55000,360.00000,8966.988242,3.191190e+06,355.882026
7,2015-12-02,362.71000,362.7100,348.15000,358.58000,5686.564000,2.020884e+06,355.378824
8,2015-12-03,360.00000,369.9900,356.52000,361.77000,7242.504373,2.631859e+06,363.390713
9,2015-12-03,359.11000,370.5300,356.21000,361.42000,12190.883936,4.423951e+06,362.890080


In [5]:
# open, high, low, close, and weighted price are averaged by date from all exchanges 
df_final = df_total.groupby('Date').mean()
df_final.drop(['Volume (BTC)','Volume (Currency)'], axis=1, inplace=True)
df_final.columns = ['open','high','low','close','weighted_price']

# high_low_spread = high - low
df_final['high_low_spread'] = df_final['high'] - df_final['low']

# midquote_price = (high-low)/2
df_final['midquote_price'] = df_final['high_low_spread']/2

# effective_spread = 2 * (close - midquote_price)
df_final['effective_spread'] = 2 * (df_final['close'] - df_final['midquote_price'])


# closing trailing 7 days
df_final["7_trailing"] = ""
# closing next trailing 7 days
df_final["7_next_trailing"] = ""

for i in range(len(df_final['close'])):
    print(i)
    try:
        df_final.set_value(df_final.index.values[i],'7_trailing',df_final['close'][i] - df_final['close'][i-7])
        df_final.set_value(df_final.index.values[i],'7_next_trailing',df_final['close'][i+7] - df_final['close'][i])
    except:
        print('index {} fail'.format(i))
    
df_final = df_final.round(2)

# volume (btc and currency) are sum by date from all exchanges
df_final = pd.concat([df_final, df_total.groupby('Date').sum()[['Volume (BTC)','Volume (Currency)']]], axis=1, 
                   join_axes=[df_final.index])
df_final = df_final.round({'Volume (Currency)': 2})
df_final.rename(columns = {'Volume (BTC)':'volume_btc','Volume (Currency)':'volume_currency'},inplace=True)

df_final.head(10)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,open,high,low,close,weighted_price,high_low_spread,midquote_price,effective_spread,7_trailing,7_next_trailing,volume_btc,volume_currency
Date,,,,,,,,,,,,
2015-12-01,377.07,378.84,354.81,362.66,364.76,24.03,12.02,701.29,-12426.1,55.0145,31933.473982,11649167.21
2015-12-02,362.62,362.75,348.39,359.04,355.67,14.37,7.18,703.71,-11171.8,58.27,27225.516220,9683443.68
2015-12-03,359.46,370.16,356.45,361.46,363.09,13.71,6.86,709.21,-10427.6,54.0905,24409.082265,8860908.00
2015-12-04,362.27,364.53,356.06,364.01,361.01,8.48,4.24,719.54,-10484.2,86.4961,18834.727468,6790670.49
2015-12-05,363.86,390.18,363.60,387.74,376.55,26.58,13.29,748.89,-11002.2,47.2825,30731.363028,11588928.33
2015-12-06,389.09,401.66,382.36,387.98,392.91,19.30,9.65,756.67,-10754.9,44.7163,29450.487297,11551999.37
2015-12-07,387.67,398.84,382.69,395.18,392.45,16.15,8.07,774.21,-10404.4,48.1975,27599.172527,10837927.67
2015-12-08,394.50,417.92,388.14,417.67,397.93,29.77,14.89,805.57,55.0145,47.165,24513.171674,9775048.27
2015-12-09,416.88,423.29,401.92,417.31,414.90,21.37,10.69,813.25,58.27,37.5822,40924.212532,16985622.56


In [6]:
df_final.reset_index(inplace=True)
df_final = df_final.loc[(df_final['Date'] >= datetime(2016, 1, 1)) & (df_final['Date'] < datetime(2018,1,1))]
df_final.set_index('Date', inplace=True)

In [7]:
df_final

,open,high,low,close,weighted_price,high_low_spread,midquote_price,effective_spread,7_trailing,7_next_trailing,volume_btc,volume_currency
Date,,,,,,,,,,,,
2016-01-01,431.35,435.88,428.22,433.66,432.87,7.66,3.83,859.65,-22.5262,20.2162,8749.673805,3.789079e+06
2016-01-02,434.71,435.99,431.18,434.44,433.70,4.81,2.41,864.07,16.345,15.4125,6591.310053,2.862465e+06
2016-01-03,434.02,434.53,424.06,429.88,429.01,10.47,5.24,849.28,5.965,19.125,9678.402186,4.157152e+06
2016-01-04,430.43,435.12,428.82,433.01,432.40,6.30,3.15,859.72,10.8135,16.5891,13463.971066,5.832899e+06
2016-01-05,432.75,435.14,429.50,432.35,432.15,5.64,2.82,859.06,0.085,3.425,10381.184703,4.492484e+06
2016-01-06,432.20,432.44,427.21,429.90,430.16,5.23,2.62,854.56,3.98282,3.02542,12809.357391,5.519290e+06
2016-01-07,429.76,459.12,429.25,458.12,447.85,29.87,14.94,886.37,27.4959,-27.9421,34986.550228,1.570127e+07
2016-01-08,457.84,463.33,446.58,453.88,454.98,16.75,8.37,891.00,20.2162,-93.27,21181.179401,9.636611e+06
2016-01-09,453.13,456.18,447.84,449.85,451.66,8.34,4.17,891.36,15.4125,-61.6755,9862.746028,4.446701e+06


In [8]:
df_final.to_csv('data_final.csv', sep='\t', encoding='utf-8')

In [10]:
df_final.tail()

,open,high,low,close,weighted_price,high_low_spread,midquote_price,effective_spread,7_trailing,7_next_trailing,volume_btc,volume_currency
Date,,,,,,,,,,,,
2017-12-27,15786.90,16506.33,14543.49,15396.67,15487.13,1962.84,981.42,28830.50,-1017.97,-252.548,45864.771929,7.107715e+08
2017-12-28,15397.34,15500.79,13560.62,14483.22,14248.84,1940.17,970.08,27026.28,-976.555,702.438,56110.855923,7.993687e+08
2017-12-29,14480.40,15119.53,14075.60,14448.19,14569.39,1043.93,521.97,27852.45,409.957,2502.24,37483.745014,5.459156e+08
2017-12-30,14450.61,14523.46,12266.14,12689.09,13329.00,2257.32,1128.66,23120.85,-2040.88,4433.07,57024.039238,7.599804e+08
2017-12-31,12688.51,14277.50,12538.27,13899.16,13376.96,1739.23,869.61,26059.08,-149.143,2249.29,34133.350349,4.578112e+08


In [11]:
import plotly.plotly as py
import plotly.graph_objs as go 

from datetime import datetime
import pandas_datareader.data as web

In [12]:
df = df_final

In [35]:
import quandl, math
import numpy as np
import pandas as pd
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from matplotlib import style
import datetime
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [36]:


trace_high = go.Scatter(x=df.index,
                        y=df.high,
                        name='High',
                        line=dict(color='#33CFA5'))

trace_high_avg = go.Scatter(x=df.index,
                            y=[df.high.mean()]*len(df.index),
                            name='High Average',
                            visible=False,
                            line=dict(color='#33CFA5', dash='dash'))

trace_low = go.Scatter(x=df.index,
                       y=df.low,
                       name='Low',
                       line=dict(color='#F06A6A'))

trace_low_avg = go.Scatter(x=df.index,
                           y=[df.low.mean()]*len(df.index),
                           name='Low Average',
                           visible=False,
                           line=dict(color='#F06A6A', dash='dash'))

data = [trace_high, trace_high_avg, trace_low, trace_low_avg]

high_annotations=[dict(x='2016-03-01',
                       y=df.high.mean(),
                       xref='x', yref='y',
                       text='High Average:<br>'+str(df.high.mean()),
                       ax=0, ay=-40),
                  dict(x=df.high.idxmax(),
                       y=df.high.max(),
                       xref='x', yref='y',
                       text='High Max:<br>'+str(df.high.max()),
                       ax=0, ay=-40)]
low_annotations=[dict(x='2016-03-01',
                      y=df.low.mean(),
                      xref='x', yref='y',
                      text='Low Average:<br>'+str(df.low.mean()),
                      ax=0, ay=40),
                 dict(x=df.high.idxmin(),
                      y=df.low.min(),
                      xref='x', yref='y',
                      text='Low Min:<br>'+str(df.low.min()),
                      ax=0, ay=40)]

updatemenus = list([
    dict(type="buttons",
         active=-1,
         buttons=list([   
            dict(label = 'High',
                 method = 'update',
                 args = [{'visible': [True, True, False, False]},
                         {'title': 'Bitcoin High',
                          'annotations': high_annotations}]),
            dict(label = 'Low',
                 method = 'update',
                 args = [{'visible': [False, False, True, True]},
                         {'title': 'Bitcoin Low',
                          'annotations': low_annotations}]),
            dict(label = 'Both',
                 method = 'update',
                 args = [{'visible': [True, True, True, True]},
                         {'title': 'Bitcoin',
                          'annotations': high_annotations+low_annotations}]),
            dict(label = 'Reset',
                 method = 'update',
                 args = [{'visible': [True, False, True, False]},
                         {'title': 'Bitcoin',
                          'annotations': []}])
        ]),
    )
])

layout = dict(title='Bitcoin Prices', showlegend=False,
              updatemenus=updatemenus)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='update_button')

In [27]:
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
print (__version__) # requires version >= 1.9.0

import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go

import pandas as pd

2.0.11


In [29]:

trace_high = go.Scatter(x=df.index,
                        y=df.high,
                        name='High',
                        line=dict(color='#33CFA5'))

trace_high_avg = go.Scatter(x=df.index,
                            y=[df.high.mean()]*len(df.index),
                            name='High Average',
                            visible=False,
                            line=dict(color='#33CFA5', dash='dash'))

trace_low = go.Scatter(x=df.index,
                       y=df.low,
                       name='Low',
                       line=dict(color='#F06A6A'))

trace_low_avg = go.Scatter(x=df.index,
                           y=[df.low.mean()]*len(df.index),
                           name='Low Average',
                           visible=False,
                           line=dict(color='#F06A6A', dash='dash'))

data = [trace_high, trace_high_avg, trace_low, trace_low_avg]

high_annotations=[dict(x='2016-03-01',
                       y=df.high.mean(),
                       xref='x', yref='y',
                       text='High Average:<br>'+str(df.high.mean()),
                       ax=0, ay=-40),
                  dict(x=df.high.idxmax(),
                       y=df.high.max(),
                       xref='x', yref='y',
                       text='High Max:<br>'+str(df.high.max()),
                       ax=0, ay=-40)]
low_annotations=[dict(x='2015-05-01',
                      y=df.low.mean(),
                      xref='x', yref='y',
                      text='Low Average:<br>'+str(df.low.mean()),
                      ax=0, ay=40),
                 dict(x=df.high.idxmin(),
                      y=df.low.min(),
                      xref='x', yref='y',
                      text='Low Min:<br>'+str(df.low.min()),
                      ax=0, ay=40)]

updatemenus = list([
    dict(type="buttons",
         active=-1,
         buttons=list([   
            dict(label = 'High',
                 method = 'update',
                 args = [{'visible': [True, True, False, False]},
                         {'title': 'Bitcoin High',
                          'annotations': high_annotations}]),
            dict(label = 'Low',
                 method = 'update',
                 args = [{'visible': [False, False, True, True]},
                         {'title': 'Bitcoin Low',
                          'annotations': low_annotations}]),
            dict(label = 'Both',
                 method = 'update',
                 args = [{'visible': [True, True, True, True]},
                         {'title': 'Bitcoin',
                          'annotations': high_annotations+low_annotations}]),
            dict(label = 'Reset',
                 method = 'update',
                 args = [{'visible': [True, False, True, False]},
                         {'title': 'Bitcoin Prices',
                          'annotations': []}])
        ]),
    )
])

layout = dict(title='Bitcoin Prices', showlegend=False,
              updatemenus=updatemenus)



In [31]:
fig = dict(data=data, layout=layout)
plot(fig, filename='update_button')

C:\Users\Luisk8nj\Documents\Anaconda\envs\PythonData\lib\site-packages\plotly\offline\offline.py:459: UserWarning:

Your filename `update_button` didn't end with .html. Adding .html to the end of your file.



'file://C:\\Users\\Luisk8nj\\Desktop\\bitcoin_data_analaysis\\jamie\\update_button.html'